In [1]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [ ]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

In [3]:
list(dataset_dir.iterdir())

[WindowsPath('/tmp/.keras/README.md'),
 WindowsPath('/tmp/.keras/stack_overflow_16k.tar.gz'),
 WindowsPath('/tmp/.keras/test'),
 WindowsPath('/tmp/.keras/train')]

In [4]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[WindowsPath('/tmp/.keras/train/csharp'),
 WindowsPath('/tmp/.keras/train/java'),
 WindowsPath('/tmp/.keras/train/javascript'),
 WindowsPath('/tmp/.keras/train/python')]

In [5]:
sample_file = train_dir/'python/1755.txt'

with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [6]:
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [51]:
type(raw_train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [49]:
raw_train_ds

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [12]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question: ", text_batch.numpy()[i][:20] + b"...")
    print("Label:", label_batch.numpy()[i])

Question:  b'"function expected e...'
Label: 2
Question:  b'"find highest double...'
Label: 1
Question:  b'"exception of type \'...'
Label: 0
Question:  b'why does print() on ...'
Label: 3
Question:  b'"can i use node.chil...'
Label: 2
Question:  b'"how do i convert a ...'
Label: 3
Question:  b'"find index of a str...'
Label: 1
Question:  b'"blank code not work...'
Label: 2
Question:  b'"ifs and arraylists ...'
Label: 1
Question:  b'blank annotation ele...'
Label: 1


In [8]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [13]:
# Create a validation set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [24]:
test_dir = dataset_dir/'test'

# Create a test set.
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [16]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [17]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [18]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [22]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"can i use node.childnodes in the if clause? can i do..if (node.childnodes) {.  // do something.}...instead of ..if (node.haschildnodes()) {.  // do something.}...i tried it, and it works. i tried !div.childnodes on a div element with stuff inside it and this returned the boolean false. it looks like it\'s working, but are there any traps that i\'m overlooking here?"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [52]:
type(raw_train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [25]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [26]:
binary_model = tf.keras.Sequential([layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 6s 27ms/step - loss: 1.1184 - accuracy: 0.6559 - val_loss: 0.9154 - val_accuracy: 0.7756
Epoch 2/10
200/200 [==============================] - 6s 27ms/step - loss: 0.7781 - accuracy: 0.8177 - val_loss: 0.7508 - val_accuracy: 0.7969
Epoch 3/10
200/200 [==============================] - 6s 28ms/step - loss: 0.6265 - accuracy: 0.8642 - val_loss: 0.6657 - val_accuracy: 0.8125
Epoch 4/10
200/200 [==============================] - 13s 65ms/step - loss: 0.5333 - accuracy: 0.8850 - val_loss: 0.6110 - val_accuracy: 0.8281
Epoch 5/10
200/200 [==============================] - 8s 38ms/step - loss: 0.4675 - accuracy: 0.9031 - val_loss: 0.5749 - val_accuracy: 0.8331
Epoch 6/10
200/200 [==============================] - 7s 31ms/step - loss: 0.4173 - accuracy: 0.9177 - val_loss: 0.5477 - val_accuracy: 0.8369
Epoch 7/10
200/200 [==============================] - 6s 28ms/step - loss: 0.3773 - accuracy: 0.9281 - val_loss: 0.5279 - val_accuracy: 0.838

In [33]:
binary_model1 = tf.keras.Sequential([
    layers.Dense(6),
    layers.Dense(5),
    layers.Dense(4)
])

binary_model1.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model1.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 6s 27ms/step - loss: 0.9573 - accuracy: 0.6628 - val_loss: 0.6393 - val_accuracy: 0.7869
Epoch 2/10
200/200 [==============================] - 6s 30ms/step - loss: 0.4544 - accuracy: 0.8559 - val_loss: 0.4864 - val_accuracy: 0.8231
Epoch 3/10
200/200 [==============================] - 6s 27ms/step - loss: 0.2682 - accuracy: 0.9214 - val_loss: 0.4606 - val_accuracy: 0.8363
Epoch 4/10
200/200 [==============================] - 6s 27ms/step - loss: 0.1630 - accuracy: 0.9606 - val_loss: 0.4797 - val_accuracy: 0.8369
Epoch 5/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0995 - accuracy: 0.9811 - val_loss: 0.5170 - val_accuracy: 0.8338
Epoch 6/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0614 - accuracy: 0.9912 - val_loss: 0.5682 - val_accuracy: 0.8313
Epoch 7/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0384 - accuracy: 0.9959 - val_loss: 0.6125 - val_accuracy: 0.8294

In [27]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [28]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 12s 51ms/step - loss: 1.1343 - accuracy: 0.5184 - val_loss: 0.7427 - val_accuracy: 0.7106
Epoch 2/5
200/200 [==============================] - 10s 47ms/step - loss: 0.6086 - accuracy: 0.7650 - val_loss: 0.5412 - val_accuracy: 0.7944
Epoch 3/5
200/200 [==============================] - 10s 50ms/step - loss: 0.3634 - accuracy: 0.8852 - val_loss: 0.4818 - val_accuracy: 0.8094
Epoch 4/5
200/200 [==============================] - 10s 51ms/step - loss: 0.1967 - accuracy: 0.9555 - val_loss: 0.4833 - val_accuracy: 0.8194
Epoch 5/5
200/200 [==============================] - 10s 48ms/step - loss: 0.0954 - accuracy: 0.9837 - val_loss: 0.5056 - val_accuracy: 0.8138


In [29]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 6s 21ms/step - loss: 0.5266 - accuracy: 0.8096
Binary model accuracy: 81.39%
Int model accuracy: 80.96%


In [34]:
binary_model1.evaluate(binary_test_ds)

250/250 [==============================] - 5s 19ms/step - loss: 0.8412 - accuracy: 0.7871


[0.8411602973937988, 0.7871249914169312]

In [35]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))

250/250 [==============================] - 5s 19ms/step - loss: 0.5187 - accuracy: 0.8139 0s - loss: 0.5182 - accu
Accuracy: 81.39%


In [36]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [37]:
inputs = [
    "how do I extract keys from a dict into a list?",  # 'python'
    "debug public static void main(string[] args) {...}",  # 'java'
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

Question:  how do I extract keys from a dict into a list?
Predicted label:  b'python'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'


# Exemplo 2
Irrelevante